<a href="https://colab.research.google.com/github/Nell87/drivendata_richter/blob/main/script/Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Model (Random Forest)
This first model will be used as comparison for the performance of next ones. 

In [7]:
####    INCLUDES  _______________________________________ #### 
#Loading Libraries:# 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier



####    READING TRAIN AND TEST DATA _______________________________________ #### 
train= data = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_values.csv")
train_labels = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/train_labels.csv")
# train = train.merge(train_labels, on='building_id')
test = pd.read_csv("https://raw.githubusercontent.com/Nell87/drivendata_richter/main/data/test_values.csv")

In [8]:
####    PREPROCESSING DATA - SCALING NUMERICAL DATA  _______________________________________ #### 
# Numerical columns
num_col = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'age', 'area_percentage','count_floors_pre_eq' ,'height_percentage']

# Define standard scaler
scaler = StandardScaler()

# Transform data 
train[num_col] = scaler.fit_transform(train[num_col])

####    PREPROCESSING DATA - DUMMY VARIABLES  _______________________________________ #### 
train = pd.get_dummies(train.drop('damage_grade', axis=1))

KeyError: ignored

In [ ]:
# RF
rf_clf = RandomForestClassifier() 

#Train the model using the training sets
rf_clf.fit(train,train_labels)